# Supply Network Design 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vitostamatti/mathematical-optimization-pyomo/blob/main/notebooks/04-supply-chain-design-01.ipynb)

In this problem, we have six end customers, each with a known demand for a product.  Customer demand can be satisfied from a set of six depots, or directly from a set of two factories.  Each depot can support a maximum volume of product moving through it, and each factory can produce a maximum amount of product.  There are known costs associated with transporting the product, from a factory to a depot, from a depot to a customer, or from a factory directly to a customer. This extension provides the opportunity to choose which four of the six possible depots to open.  It also provides an option of expanding capacity at one specific depot.

Our supply network has two factories, in Liverpool and Brighton, that produce a product.  Each has a maximum production capacity:

| Factory | Supply (tons) |
| --- | --- |
| Liverpool | 150,000 |
| Brighton |  200,000 |

The product can be shipped from a factory to a set of six depots.  Each depot has a maximum throughput.  Depots don't produce or consume the product; they simply pass the product through to customers.

| Depot | Throughput (tons) |
| --- | --- |
| Newcastle | 70,000 |
| Birmingham | 50,000 |
| London | 100,000 |
| Exeter | 40,000 |
| Bristol | 30,000 |
| Northampton | 25,000 |

We can actually only choose four of the six depots to open.  Opening a depot has a cost:

| Depot | Cost to open |
| --- | --- |
| Newcastle | 10,000 |
| Exeter | 5,000 |
| Bristol | 12,000 |
| Northampton | 4,000 |


Our network has six customers, each with a given demand.

| Customer | Demand (tons) |
| --- | --- |
| C1 | 50,000 |
| C2 | 10,000 |
| C3 | 40,000 |
| C4 | 35,000 |
| C5 | 60,000 |
| C6 | 20,000 |

Shipping costs are given in the following table (in dollars per ton).  Columns are source cities and rows are destination cities.  Thus, for example, it costs $1 per ton to ship the product from Liverpool to London.  A '-' in the table indicates that that combination is not possible, so for example it is not possible to ship from the factory in Brighton to the depot in Newcastle.

| To | Liverpool | Brighton | Newcastle | Birmingham | London | Exeter | Briston | Northhampton
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Depots |
| Newcastle   | 0.5 |   - |
| Birmingham  | 0.5 | 0.3 |
| London      | 1.0 | 0.5 |
| Exeter      | 0.2 | 0.2 |
| Bristol     | 0.6 | 0.4 |
| Northampton | 0.4 | 0.3 |
| Customers |
| C1 | 1.0 | 2.0 |   - | 1.0 |   - |   - | 1.2 |   - |
| C2 |   - |   - | 1.5 | 0.5 | 1.5 |   - | 0.6 | 0.4 |
| C3 | 1.5 |   - | 0.5 | 0.5 | 2.0 | 0.2 | 0.5 |   - |
| C4 | 2.0 |   - | 1.5 | 1.0 |   - | 1.5 |   - | 0.5 |
| C5 |   - |   - |   - | 0.5 | 0.5 | 0.5 | 0.3 | 0.6 |
| C6 | 1.0 |   - | 1.0 |   - | 1.5 | 1.5 | 0.8 | 0.9 |

The questions to be answered: (i) Which four depots should be opened? (ii) Should Birmingham be expanded? (iii) Which depots should be used to satisfy customer demand?

---


## Model Formulation

### Sets and Indices

$f \in \text{Factories}=\{\text{Liverpool}, \text{Brighton}\}$

$d \in \text{Depots}=\{\text{Newcastle}, \text{Birmingham}, \text{London}, \text{Exeter}, \text{Bristol}, \text{Northampton}\}$

$c \in \text{Customers}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}$

$\text{Cities} = \text{Factories} \cup \text{Depots} \cup \text{Customers}$

### Parameters

$\text{cost}_{s,t} \in \mathbb{R}^+$: Cost of shipping one ton from source $s$ to destination $t$.

$\text{supply}_f \in \mathbb{R}^+$: Maximum possible supply from factory $f$ (in tons).

$\text{through}_d \in \mathbb{R}^+$: Maximum possible flow through depot $d$ (in tons).

$\text{demand}_c \in \mathbb{R}^+$: Demand for goods at customer $c$ (in tons).

$\text{opencost}_d \in \mathbb{R}^+$: Cost of opening depot $d$ (in dollars).

### Decision Variables

$\text{flow}_{s,t} \in \mathbb{N}^+$: Quantity of goods (in tons) that is shipped from source $s$ to destionation $t$.

$\text{open}_{d} \in [0,1]$: Is depot $d$ open?



### Objective Function

- **Cost**: Minimize total shipping costs plus costs of opening depots.

\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{s,t}*\text{flow}_{s,t}} +
                          \sum_{{d} \in \text{Depots}}{\text{opencost}_d*\text{open}_d}
\end{equation}

### Constraints

- **Factory output**: Flow of goods from a factory must respect maximum capacity.

\begin{equation}
\sum_{t \in \text{Cities}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Factories}
\end{equation}

- **Customer demand**: Flow of goods must meet customer demand.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Customers}
\end{equation}

- **Depot flow**: Flow into a depot equals flow out of the depot.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} = 
\sum_{t \in \text{Cities}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}

- **Depot capacity (all but Birmingham)**: Flow into a depot must respect depot capacity, and is only allowed if the depot is open.

\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} \leq \text{through}_{d} * \text{open}_{d}
\quad \forall d \in \text{Depots} - \text{Birmingham}
\end{equation}

- **Depot capacity (Birmingham)**: Flow into Birmingham must respect depot capacity, which may have been expanded.

\begin{equation}
\sum_{s \in \text{Cities}} \text{flow}_{s,\text{Birmingham}} \leq \text{through}_{\text{Birmingham}} + 20000 * \text{expand}
\end{equation}

- **Open depots**: At most 4 open depots (no choice for Birmingham or London).

\begin{equation}
\sum_{d \in \text{Depots}}{\text{open}_{d}} \leq 4
\end{equation}

\begin{equation}
\text{open}_{\text{Birmingham}} = \text{open}_{\text{London}} = 1
\end{equation}


In [13]:
import pandas as pd

supply = dict({'Liverpool': 150000,
               'Brighton': 200000})

df_supply = pd.DataFrame.from_dict(supply, orient='index', columns=['supply'])
df_supply

,supply
Liverpool,150000
Brighton,200000


In [14]:
through = dict({'Newcastle': 70000,
                'Birmingham': 50000,
                'London': 100000,
                'Exeter': 40000,
                'Bristol': 30000,
                'Northampton': 25000})

df_through = pd.DataFrame.from_dict(through, orient='index', columns=['through'])
df_through

,through
Newcastle,70000
Birmingham,50000
London,100000
Exeter,40000
Bristol,30000
Northampton,25000


In [93]:

opencost = dict({'Newcastle': 10000,
                 'Birmingham': 0,
                 'London': 0,
                 'Exeter': 5000,
                 'Bristol': 12000,
                 'Northampton': 4000
                 })
df_open_cost = pd.DataFrame.from_dict(opencost, orient='index', columns=['open_cost'])
df_open_cost

,open_cost
Newcastle,10000
Birmingham,0
London,0
Exeter,5000
Bristol,12000
Northampton,4000


In [63]:

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

df_demand = pd.DataFrame.from_dict(demand, orient='index', columns=['demand'])
df_demand


,demand
C1,50000
C2,10000
C3,40000
C4,35000
C5,60000
C6,20000


In [37]:
# Create dictionaries to capture factory supply limits, depot throughput limits, cost of opening depots, and customer demand.

# Create a dictionary to capture shipping costs.

arcs_cost = dict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'Bristol'): 0.6,
    ('Liverpool', 'Northampton'): 0.4,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'Bristol'): 0.4,
    ('Brighton', 'Northampton'): 0.3,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5,
    ('Bristol', 'C1'): 1.2,
    ('Bristol', 'C2'): 0.6,
    ('Bristol', 'C3'): 0.5,
    ('Bristol', 'C5'): 0.3,
    ('Bristol', 'C6'): 0.8,
    ('Northampton', 'C2'): 0.4,
    ('Northampton', 'C4'): 0.5,
    ('Northampton', 'C5'): 0.6,
    ('Northampton', 'C6'): 0.9
})

df_arcs_cost= pd.DataFrame(
    arcs_cost.values(),index=pd.MultiIndex.from_tuples(arcs_cost.keys()), columns=['cost'])
# df_arcs_cost


In [55]:
df_arcs_cost


cost
Liverpool   Newcastle     0.5
            Birmingham    0.5
            London        1.0
            Exeter        0.2
            Bristol       0.6
            Northampton   0.4
            C1            1.0
            C3            1.5
            C4            2.0
            C6            1.0
Brighton    Birmingham    0.3
            London        0.5
            Exeter        0.2
            Bristol       0.4
            Northampton   0.3
            C1            2.0
Newcastle   C2            1.5
            C3            0.5
            C5            1.5
            C6            1.0
Birmingham  C1            1.0
            C2            0.5
            C3            0.5
            C4            1.0
            C5            0.5
London      C2            1.5
            C3            2.0
            C5            0.5
            C6            1.5
Exeter      C3            0.2
            C4            1.5
            C5            0.5
            C6            1.5
Bristol     C1            1.2
            C2            0.6
            C3            0.5
            C5            0.3
            C6            0.8
Northampton C2            0.4
            C4            0.5
            C5            0.6
            C6            0.9

In [41]:
import pyomo.environ as pyo


,supply
Liverpool,150000
Brighton,200000


In [94]:
m = pyo.ConcreteModel()

# SETS
m.F = pyo.Set(initialize=df_supply.index)

m.D = pyo.Set(initialize=df_through.index)
m.C = pyo.Set(initialize=df_demand.index)
m.N = m.F | m.D | m.C # union
m.NN = pyo.Set(initialize=m.N*m.N, filter=lambda m,i,j: (i,j) in df_arcs_cost.index)

m.FN = pyo.Set(initialize=m.F*m.N) & m.NN # intersection
m.NC = pyo.Set(initialize=m.N*m.C) & m.NN # intersection
m.ND = pyo.Set(initialize=m.N*m.D) & m.NN # intersection
m.DN = pyo.Set(initialize=m.D*m.N) & m.NN # intersection

m.flow = pyo.Var(m.NN, domain=pyo.NonNegativeReals)

m.open_cost = pyo.Var(domain=pyo.Reals)
m.flow_cost = pyo.Var(domain=pyo.Reals)

m.open = pyo.Var(m.D, domain=pyo.Binary) #realizar o no las OT

# Initially open depots
m.open["Birmingham"].setlb(1)
m.open["London"].setlb(1)

#RESTRICCIONES

def c_factory_capacity(m, f):
    return sum(
        m.flow[f,n] for n in m.N
        if (f,n) in m.FN
    ) <= df_supply.loc[f,'supply']

m.c_factory_capacity = pyo.Constraint(m.F,rule = c_factory_capacity) 


def c_customer_demand(m, c):
    return sum(
        m.flow[n,c] for n in m.N
        if (n,c) in m.NC
    ) == df_demand.loc[c,'demand']

m.c_customer_demand = pyo.Constraint(m.C, rule = c_customer_demand) 


def c_depot_balance(m, d):
    return sum(
        m.flow[n,d] for n in m.N
        if (n,d) in m.ND
    ) == sum(
        m.flow[d,n] for n in m.N
        if (d,n) in m.DN        
    )

m.c_depot_balance = pyo.Constraint(m.D, rule = c_depot_balance) 


def c_depot_capacity(m, d):
    return sum(
        m.flow[n,d] for n in m.N
        if (n,d) in m.ND
    ) <= df_through.loc[d,'through']*m.open[d]

m.c_depot_capacity = pyo.Constraint(m.D, rule = c_depot_capacity) 


def c_max_open_depots(m):
    return sum(m.open[d] for d in m.D) <= 4

m.c_max_open_depots = pyo.Constraint(rule = c_max_open_depots) 


def c_open_cost(m):
    return  m.open_cost == sum(
        df_open_cost.loc[d,'open_cost']*m.open[d]
        for d in m.D
    )

m.c_open_cost = pyo.Constraint(rule = c_open_cost) 


def c_flow_cost(m):
    return  m.flow_cost == sum(
        df_arcs_cost.loc[(i,j),'cost']*m.flow[i,j]
        for i,j in m.NN
    )

m.c_flow_cost = pyo.Constraint(rule = c_flow_cost) 


def OBJ(m): 
    return m.flow_cost + m.open_cost

m.OBJ = pyo.Objective(rule = OBJ, sense = pyo.minimize)


In [106]:
# pyo.SolverFactory('../cbc-win64/cbc').solve(m).write() 
pyo.SolverFactory('glpk').solve(m).write() 

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 190000.0
  Upper bound: 190000.0
  Number of objectives: 1
  Number of constraints: 24
  Number of variables: 51
  Number of nonzeros: 156
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.360396146774292
# ----------------------------------------------------------
#   Solution Information
# -----------------------------

In [107]:
print("Open Depots: ",[d for d in m.D if m.open[d].value>0.5])

Open Depots:  ['Birmingham', 'London', 'Exeter', 'Northampton']


In [108]:
product_flow = pd.DataFrame(columns=["From", "To", "Flow"])

for i,j in m.NN:
    if m.flow[i,j].value > 1e-6:
        product_flow = product_flow.append({
            "From": i, "To": j, "Flow": m.flow[i,j].value
            }, ignore_index=True)  
product_flow.index=[''] * len(product_flow)
product_flow

,From,To,Flow
,Liverpool,Exeter,40000.0
,Liverpool,C1,50000.0
,Liverpool,C6,20000.0
,Brighton,Birmingham,50000.0
,Brighton,London,30000.0
,Brighton,Northampton,25000.0
,Birmingham,C2,10000.0
,Birmingham,C4,10000.0
,Birmingham,C5,30000.0
,London,C5,30000.0
